
# 第9章 もっと入力、もっと出力

```zsh
$ghc --make capslocker.hs 
[1 of 1] Compiling Main             ( capslocker.hs, capslocker.o )
Linking capslocker ...

$./capslocker < haiku.txt   
I'M A LIL' TEAPOT
WHAT'S WITH THAT AIRPLANE FOOD, HUH?
IT'S SO SMALL, TASTELESS
capslocker: <stdin>: hGetLine: end of file
```

In [2]:
shortLinesOnly :: String -> String
shortLinesOnly = unlines . filter (\line -> length line < 10) . lines

In [3]:
:t unlines

unlines :: [String] -> String

In [4]:
main = interact shortLinesOnly

In [5]:
respondPalindromes :: String -> String
respondPalindromes = unlines . map (\xs -> if isPal xs then "palindrome" else "not a palindrome") . lines

isPal :: String -> Bool
isPal xs = xs == reverse xs

In [6]:
import System.IO

main = do
    handle <- openFile "baabaa.txt" ReadMode
    contents <- hGetContents handle
    putStr contents
    hClose handle

In [7]:
main

Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;

In [8]:
:t openFile

openFile :: FilePath -> IOMode -> IO Handle

In [9]:
:t FilePath

Not in scope: data constructor `FilePath'

In [10]:
:k FilePath

FilePath :: *

In [11]:
import System.IO

main = do
    withFile "baabaa.txt" ReadMode $ \handle -> do
        contents <- hGetContents handle
        putStr contents
        

Line 1: Redundant do
Found:
do withFile "baabaa.txt" ReadMode $
     \ handle ->
       do contents <- hGetContents handle
          putStr contents
Why not:
withFile "baabaa.txt" ReadMode $
  \ handle ->
    do contents <- hGetContents handle
       putStr contents

In [12]:
main

Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;

次回  

### ブラケットの時間

デュエルスタンバイ

Control.Exception.bracketの型シグネチャ

```
bracket :: IO a -> (a -> IO b) -> (a -> IO c) -> IO c
```

In [13]:
import Control.Exception

withFile :: FilePath -> IOMode -> (Handle -> IO a) -> IO a
withFile name mode f = bracket (openFile name mode)
    (\handle -> hClose handle)
    (\handle -> f handle)

Line 3: Avoid lambda
Found:
\ handle -> hClose handle
Why not:
hCloseLine 4: Avoid lambda
Found:
\ handle -> f handle
Why not:
f

In [14]:
import System.IO

main = do
    contents <- readFile "baabaa.txt"
    putStr contents
    

In [15]:
main

Baa, baa, black sheep,
Have you any wool?
Yes, sir, yes, sir,
Three bags full;

In [16]:
import System.IO
import Data.Char

main = do
    contents <- readFile "baabaa.txt"
    writeFile "baabaacaps.txt" (map toUpper contents)

main

## ToDoリスト

In [18]:
import System.IO

main = do
    todoItem <- getLine
    appendFile "todo.txt" (todoItem ++ "\n")

In [19]:
main

hoge


In [20]:
main

fuga


In [21]:
do
    contents <- readFile "todo.txt"
    putStr contents

hoge
fuga

In [1]:
import System.IO
import System.Directory
import Data.List

main = do
    contents <- readFile "todo.txt"
    let todoTasks = lines contents
        numberedTasks = zipWith (\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStrLn "These are your TO-DO items:"
    mapM_ putStrLn numberedTasks
    putStrLn "Which one do you want to delete?"
    numberString <- getLine
    let number = read numberString
        newTodoItems = unlines $ delete (todoTasks !! number) todoTasks
    (tempName, tempHandle) <- openTempFile "." "temp"
    hPutStr tempHandle newTodoItems
    hClose tempHandle
    removeFile "todo.txt"
    renameFile tempName "todo.txt"

In [ ]:
main

These are your TO-DO items:
0 - hoge
1 - fuga
Which one do you want to delete?

!! 添字に対応するリストの要素を返す  
delete リストから指定した要素のうち最初に出るものを削除した新しいリストを返す


## クリーンアップ

In [2]:
import System.IO
import System.Directory
import Data.List
import Control.Exception

main = do
    contents <- readFile "todo.txt"
    let todoTasks = lines contents
        numberedTasks = zipWith (\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStrLn "These are your TO-DO items:"
    mapM_ putStrLn numberedTasks
    putStrLn "Which one do you want to delete?"
    numberString <- getLine
    let number = read numberString
        newTodoItems = unlines $ delete (todoTasks !! number) todoTasks
    bracketOnError (openTempFile "." "temp")
        (\(tempName, tempHandle) -> do
            hClose tempHandle
            removeFile tempName)
        (\(tempName, tempHandle) -> do
            hPutStr tempHandle newTodoItems
            hClose tempHandle
            removeFile "todo.txt"
            renameFile tempName "todo.txt"
        )
-- bracketOnError便利

In [3]:
import System.Environment
import Data.List

main = do
    args <- getArgs
    progName <- getProgName
    putStrLn "The arguments are:"
    mapM putStrLn args
    putStrLn "The program name is:"
    putStrLn progName

Line 5: Use mapM_
Found:
mapM putStrLn args
Why not:
mapM_ putStrLn args

In [4]:
main

The arguments are:
kernel
/root/.local/share/jupyter/runtime/kernel-4f65b5f1-b729-4bd3-8688-4b30ec69af7e.json
--ghclib
/root/.stack/programs/x86_64-linux/ghc-7.10.2/lib/ghc-7.10.2
The program name is:
ihaskell

In [ ]:
import System.Environment
import System.Directory
import System.IO
import Control.Exception
import Data.List

dispatch :: String -> [String] -> IO ()
dispatch "add" = add
dispatch "view" = view
dispatch "remove" = remove
dispatch command = doesntExist command

doesntExist :: String -> [String] -> IO ()
doesntExist command = putStrLn $ "The " ++ command ++ " command doesn't exist"

add :: [String] -> IO ()
add [fileName, todoItem] = appendFile fileName (todoItem ++ "\n")

view :: [String] -> IO ()
view [fileName] = do
    contents <- readFile fileName
    let todoTasks = lines contents
        numberedTasks =  zipWith (\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStr $ unlines numberedTasks
    
remove :: [String] -> IO ()
remove [fileName, numberString] = do
    contents <- readFile fileName
    let todoTasks = lines contents
        numberedTasks = zipWith (\n line -> show n ++ " - " ++ line) [0..] todoTasks
    putStrLn "These are your TO-DO items:"
    mapM_ putStrLn numberedTasks
    putStrLn "Which one do you want to delete?"
    let number = read numberString
        newTodoItems = unlines $ delete (todoTasks !! number) todoTasks
    bracketOnError (openTempFile "." "temp")
        (\(tempName, tempHandle) -> do
            hClose tempHandle
            removeFile tempName)
        (\(tempName, tempHandle) -> do
            hPutStr tempHandle newTodoItems
            hClose tempHandle
            removeFile fileName
            renameFile tempName fileName
        )

main = do
    (command:argList) <- getArgs
    dispatch command argList
    


次回

### ランダム性

デュエルスタンバイ

In [1]:
randomNumber :: Int
randomNumber = 4




System.Randomモジュール
エクスポートされているrandomの型シグネチャ


```
random :: (RandomGen g, Random a) => g -> (a, g)
```

RandomGen のインスタンスとしてStdGenという型がある  
手動で乱数ジェネレータを作るにはmkStdGenを使う  

```
mkStdGen :: Int -> StdGen
```


In [3]:
import System.Random

random (mkStdGen 100)

(-3633736515773289454,693699796 2103410263)

In [4]:
random (mkStdGen 100) :: (Int, StdGen)

(-3633736515773289454,693699796 2103410263)

In [5]:
random (mkStdGen 10)

(-2774747785423059091,1925364037 2103410263)

In [6]:
random (mkStdGen 10) :: (Float, StdGen)

(3.2916963e-2,432453652 1655838864)

In [9]:
threeCoins :: StdGen -> (Bool, Bool, Bool)
threeCoins gen =
    let (firstCoin, newGen) = random gen :: (Bool, StdGen)
        (secondCoin, newGen') = random newGen :: (Bool, StdGen)
        (thirdCoin, newGen'') =  random newGen' :: (Bool, StdGen) in (firstCoin, secondCoin, thirdCoin)

In [10]:
    threeCoins (mkStdGen 10)

(True,True,True)

In [11]:
threeCoins (mkStdGen 11)

(True,True,True)

In [12]:
threeCoins (mkStdGen 12)

(True,False,True)

In [13]:
take 5 $ randoms (mkStdGen 11) :: [Int]

[5260538044923710387,4361398698747678847,-8221315287270277529,7278185606566790575,1652507602255180489]

In [14]:
take 5 $ randoms (mkStdGen 11) :: [Bool]

[True,True,True,True,False]

In [22]:
-- randoms' :: (RandomGen g, Random a) => g -> [a]
-- randoms' gen = let (value, newGen) = (random gen :: (a, g)) in value:randoms' newGen :: (g, a)

In [30]:
-- finiteRandoms :: (RandomGen g, Random a, Num n) => n -> g -> ([a], g)
-- finiteRandoms 0 gen = ([], gen)
-- finiteRandoms n gen = 
--     let (value, newGen) = random gen :: (Int, StdGen)
--        (restOfList, finalGen) = finiteRandoms (n-1) newGen :: ([Int], StdGen) in (value:restOfList, finalGen)

動かなくて困ってるから今日はこの辺でやめる